In [37]:
#####################################################################################
run_models = True
run_k_fold = False
overleaf_metrics = False
print_all = False
is_deadline = False
#####################################################################################



####################################################################################
# ground_truth_method = 'hist'
# ground_truth_method = 'ecdf'

two_labels = True
break_window_features = True
####################################################################################




#####################################################################################
k_fold_n_splits = 5
#####################################################################################



#####################################################################################
# arousal_signals = ['PP']
# arousal_signals = ['PP', 'BR']
# arousal_signals = ['PP_HR_BR']
# arousal_signals = ['PP_HR', 'HR_BR', 'PP_BR']


# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR_BR']
arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'HR_BR', 'PP_BR', 'PP_HR_BR']
#####################################################################################



#####################################################################################
%run us_common_functions.ipynb
#####################################################################################



#####################################################################################
# ['DNN', 'LSTM']
model_name = 'LSTM'
#####################################################################################




# ####################################################################################
# ### study_combinations_to_run = list(study_combinations.keys())
# ### study_combinations_to_run = list(study_combinations.keys())[:4]
# ### study_combinations_to_run = list(study_combinations.keys())[4:]

# ### study_combinations_to_run = ['sim1', 'sim1___sim2___tt1___office_tasks']
# ### study_combinations_to_run = ['sim2', 'tt1', 'office_tasks', 'sim1___sim2', 'sim1___sim2___tt1']
# ### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
# ### study_combinations_to_run = ['sim1', 'tt1']
# ### study_combinations_to_run = ['sim1']
# ### study_combinations_to_run.remove(deadline_study)
# ###################################################################################




#####################################################################################
# All Features - Without Deadline Study
####################################################################################
four_studies = [sim1, sim2, tt1, office_tasks]
study_combinations, all_study_combinations = get_study_combinations(four_studies)
study_combinations_to_run = all_study_combinations


### study_combinations_to_run = all_study_combinations = all_study_combinations[:1]
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
### print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################














####################################################################################
epochs = 10  # 10
batch_size = 32
verbose = 1
####################################################################################


In [38]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)

# figure_path = all_studies_fig_dir
# plot_name = None






###################################################
# data_6_rolling_w15_10_scaled
# data_6_rolling_w30_10_scaled
dnn_data_file = 'data_6_rolling_w30_10_scaled.csv'

dnn_window_size = dnn_data_file[-16:-11]
dnn_window_size
###################################################


# ###################################################
# # data_6_rolling_w15_5_scaled.csv
# # data_6_rolling_w30_5_scaled.csv
# # data_6_rolling_w60_5_scaled.csv
# dnn_data_file = 'data_6_rolling_w30_5_scaled.csv'

# dnn_window_size = dnn_data_file[-15:-11]
# dnn_window_size
# ####################################################################################

'30_10'

In [39]:
all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file)
# all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file).iloc[:2000,]
all_studies_df.rename({'Perinasal_Window': 'PP_Window',
                       'Heart_Window': 'HR_Window',
                       'Breathing_Window': 'BR_Window',
                      }, axis=1, inplace=True)
all_studies_df.shape

(102334, 36)

In [40]:
all_studies_df.head(2)

,Study_Name,Subject,Study_Subject,Treatment,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,PP_Mean,PP_Median,PP_SD,PP_SS,PP_Window,HR_Mean,HR_Median,HR_SD,HR_SS,HR_Window,BR_Mean,BR_Median,BR_SD,BR_SS,BR_Window,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Gender,Gender_Male
0,sim1,2,sim1_2,CD,-0.886987,0.96269,0.332151,18.0,-0.204823,0.547659,0.367288,77.0,-0.703319,-0.043052,0.096233,0.064272,"0.0355228579351853, 0.0403412260359834, 0.0419...",0.915126,3.768182,-0.513883,518.068554,"3.5681818181818272, 4.968181818181819, 5.56818...",-1.086870,-4.366682,0.174950,607.649078,"-5.666681818181818, -5.586681818181818, -5.546...",non-arousal,neutral,non-arousal,non-arousal,non-arousal,neutral,non-arousal,0.691326,0
1,sim1,2,sim1_2,CD,-0.886987,0.96269,0.332151,18.0,-0.204823,0.547659,0.367288,77.0,-1.021681,-0.061082,0.069734,0.208791,"-0.0322886468498442, -0.0433787238649969, -0.0...",0.639667,3.368182,0.126142,373.423099,"2.76818181818183, 3.5681818181818272, 3.768181...",-0.795302,-3.366682,0.262268,368.162801,"-5.066681818181818, -4.906681818181818, -4.766...",non-arousal,neutral,neutral,neutral,non-arousal,neutral,neutral,0.691326,0


In [41]:
all_studies_df.columns

Index(['Study_Name', 'Subject', 'Study_Subject', 'Treatment', 'Age',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Mental', 'NASA_Performance',
       'NASA_Physical', 'NASA_Temporal', 'NASA_Total', 'PP_Mean', 'PP_Median',
       'PP_SD', 'PP_SS', 'PP_Window', 'HR_Mean', 'HR_Median', 'HR_SD', 'HR_SS',
       'HR_Window', 'BR_Mean', 'BR_Median', 'BR_SD', 'BR_SS', 'BR_Window',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Gender', 'Gender_Male'],
      dtype='object')

In [42]:
all_studies_df = process_ts_features(all_studies_df)
all_studies_df = create_categorical_labels(all_studies_df)

In [43]:
################################################################################
# # window_cols = ['PP_Window']
# window_cols = ['PP_Window', 'HR_Window', 'BR_Window']

# for col in window_cols:
#     for i in range(len(all_studies_df[col][0])):
#         feature_name = col[:3] + str(int(dnn_window_size[:2])-i)
#         dnn_features.append(feature_name)
#         all_studies_df[feature_name] = [window[i] for window in all_studies_df[col]]
                                                   
# all_studies_df.columns
################################################################################

In [44]:
################################################################################
# dnn_features
################################################################################

In [45]:
start_time = time.time()

# study_combinations_to_run
# all_study_combinations
for study_combination in study_combinations_to_run:   
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    # print(all_studies_df.shape, train_df.shape)
    
    model_metrics = {}

    
    for arousal_signal in arousal_signals:
        
        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        if two_labels: 
            train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != 0]
        
        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
            # print(train_df.shape)
            
            model = get_trained_dnn_model(train_df, arousal_signal)

            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):
              
                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)
                    
                    if test_study_str in all_study_combinations:
                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]
                        if two_labels:
                            test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != 0]
                            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            # print(test_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
                            
                        test_dnn_model(model, test_study_str.upper(), test_df, arousal_signal)

                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics, k_fold=False), ignore_index = True)
    
        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics
    
    
display(metrics_df)
metrics_file_name = 'dnn_deadline_' if is_deadline else 'dnn_'
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + dnn_window_size + '_v4_' + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------

Loading from saved model and weights.....

------------------------ Train Study ------------------------
Train Data - True --> 
Counter({1: 2016, -1: 1979})

Validation Data - True --> 
Counter({1: 403, -1: 396})

Validation Data - Prediction --> 
Counter({-1: 515, 1: 284})

------------------------ Test Study: SIM2 ------------------------
Test Data - True --> 
Counter({1: 658, -1: 593})

Test Data - Prediction --> 
Counter({-1: 679, 1: 572})

------------------------ Test Study: TT1 ------------------------
Test Data - True --> 
Counter({1: 287, -1: 265})

Test Data - Prediction --> 
Counter({-1: 335, 1: 217})

------------------------ Test Study: OFFICE_TASKS ------------------------
Test Data - True --> 
Counter({1: 4192, -1: 4153})

Tes

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity
0,LSTM,PP,sim1,sim2,0.843554,0.931255,0.970280,0.987955,0.840223,0.931176,0.843554,0.931255,0.877263,0.939962,0.0,0.0
1,LSTM,PP,sim1,tt1,0.843554,0.847826,0.970280,0.930721,0.840223,0.846135,0.843554,0.847826,0.877263,0.872883,0.0,0.0
2,LSTM,PP,sim1,office_tasks,0.843554,0.945357,0.970280,0.995887,0.840223,0.945207,0.843554,0.945357,0.877263,0.950763,0.0,0.0
3,LSTM,HR,sim1,sim2,0.768850,0.844771,0.946483,0.964019,0.761279,0.842422,0.768850,0.844771,0.840671,0.880535,0.0,0.0
4,LSTM,HR,sim1,tt1,0.768850,0.750871,0.946483,0.962363,0.761279,0.734905,0.768850,0.750871,0.840671,0.834108,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,LSTM,BR,sim1___sim2___tt1___office_tasks,None,0.984026,None,0.999098,None,0.984025,None,0.984026,None,0.984403,None,0.0,None
143,LSTM,PP_HR,sim1___sim2___tt1___office_tasks,None,0.950097,None,0.995618,None,0.950068,None,0.950097,None,0.954251,None,0.0,None
144,LSTM,HR_BR,sim1___sim2___tt1___office_tasks,None,0.959960,None,0.996924,None,0.959902,None,0.959960,None,0.962638,None,0.0,None
145,LSTM,PP_BR,sim1___sim2___tt1___office_tasks,None,0.976656,None,0.997950,None,0.976656,None,0.976656,None,0.977445,None,0.0,None


In [46]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))

--- Total Time 5.0 mins (0.0 hours) ---


In [47]:
# root_path = add_path(all_studies_data_dir, metrics_dir) 
# files = ['metrics_lstm_v0.csv', 'metrics_lstm_v1.csv']
# final_file_name = 'metrics_lstm_all.csv'

# concat_df(root_path, files, final_file_name)